In [ ]:
# Homework 1:

# Using chinook.db write pandas code.

# Customer Purchases Analysis:
# Find the total amount spent by each customer on purchases (considering invoices).
# Identify the top 5 customers with the highest total purchase amounts.
# Display the customer ID, name, and the total amount spent for the top 5 customers.
# Album vs. Individual Track Purchases:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

In [ ]:
# Find the total amount spent by each customer on purchases (considering invoices).

In [1]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook database
conn = sqlite3.connect("chinook.db")

# Step 2: Load customers and invoices tables
customers = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql("SELECT InvoiceId, CustomerId, Total FROM invoices", conn)

conn.close()

# Step 3: Merge customers with invoices
customer_purchases = pd.merge(customers, invoices, on="CustomerId", how="inner")

# Step 4: Calculate total amount spent per customer
total_spent = (
    customer_purchases.groupby(["CustomerId", "FirstName", "LastName"])["Total"]
    .sum()
    .reset_index()
    .sort_values(by="Total", ascending=False)
)

# Step 5: Display result
print(total_spent.head())

# (Optional) Save to Excel
total_spent.to_excel("customer_total_spent.xlsx", index=False)


    CustomerId FirstName    LastName  Total
5            6    Helena        Holý  49.62
25          26   Richard  Cunningham  47.62
56          57      Luis       Rojas  46.62
44          45  Ladislav      Kovács  45.62
45          46      Hugh    O'Reilly  45.62


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
# Identify the top 5 customers with the highest total purchase amounts.

In [ ]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook database
conn = sqlite3.connect("chinook.db")

# Step 2: Load customers and invoices tables
customers = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql("SELECT CustomerId, Total FROM invoices", conn)

conn.close()

# Step 3: Merge customers with invoices
customer_purchases = pd.merge(customers, invoices, on="CustomerId", how="inner")

# Step 4: Calculate total amount spent per customer
total_spent = (
    customer_purchases.groupby(["CustomerId", "FirstName", "LastName"])["Total"]
    .sum()
    .reset_index()
    .sort_values(by="Total", ascending=False)
)

# Step 5: Get Top 5 customers
top5_customers = total_spent.head(5)

print(top5_customers)


In [ ]:
# Display the customer ID, name, and the total amount spent for the top 5 customers.

In [ ]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook database
conn = sqlite3.connect("chinook.db")

# Step 2: Load customers and invoices tables
customers = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql("SELECT CustomerId, Total FROM invoices", conn)

conn.close()

# Step 3: Merge customers with invoices
customer_purchases = pd.merge(customers, invoices, on="CustomerId", how="inner")

# Step 4: Calculate total amount spent per customer
total_spent = (
    customer_purchases.groupby(["CustomerId", "FirstName", "LastName"])["Total"]
    .sum()
    .reset_index()
    .sort_values(by="Total", ascending=False)
)

# Step 5: Keep top 5 customers
top5


In [ ]:
# Album vs. Individual Track Purchases:
# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.

In [2]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook
conn = sqlite3.connect("chinook.db")

# Load necessary tables
invoices = pd.read_sql("SELECT InvoiceId, CustomerId FROM invoices", conn)
invoice_items = pd.read_sql("SELECT InvoiceLineId, InvoiceId, TrackId FROM invoice_items", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)

conn.close()

# Step 2: Merge invoice_items with tracks to know which album each line belongs to
invoice_tracks = invoice_items.merge(tracks, on="TrackId", how="left")

# Step 3: Count how many tracks per album exist in DB
album_track_counts = tracks.groupby("AlbumId")["TrackId"].nunique().reset_index()
album_track_counts.rename(columns={"TrackId": "AlbumTotalTracks"}, inplace=True)

# Step 4: Count how many tracks from each album were purchased in each invoice
invoice_album_counts = (
    invoice_tracks.groupby(["InvoiceId", "AlbumId"])["TrackId"]
    .nunique()
    .reset_index()
    .rename(columns={"TrackId": "TracksBought"})
)

# Merge with album totals
invoice_album_counts = invoice_album_counts.merge(album_track_counts, on="AlbumId", how="left")

# Step 5: Determine if invoice is a full album purchase
invoice_album_counts["IsFullAlbum"] = invoice_album_counts["TracksBought"] == invoice_album_counts["AlbumTotalTracks"]

# Step 6: Any invoice with at least one full album counts as an album purchase
invoice_type = invoice_album_counts.groupby("InvoiceId")["IsFullAlbum"].max().reset_index()
invoice_type["PurchaseType"] = invoice_type["IsFullAlbum"].apply(lambda x: "Album" if x else "Track")

# Step 7: Link back to customers
invoice_with_customers = invoices.merge(invoice_type[["InvoiceId", "PurchaseType"]], on="InvoiceId", how="left")

# Step 8: Determine preference per customer
customer_pref = (
    invoice_with_customers.groupby("CustomerId")["PurchaseType"]
    .apply(lambda x: "Album" if (x == "Album").sum() > (x == "Track").sum() else "Track")
    .reset_index()
)

# Step 9: Calculate percentage
total_customers = customer_pref.shape[0]
track_customers = (customer_pref["PurchaseType"] == "Track").sum()
percentage_track_pref = (track_customers / total_customers) * 100

print(f"Percentage of customers preferring individual tracks: {percentage_track_pref:.2f}%")



Percentage of customers preferring individual tracks: 100.00%


In [ ]:
# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.

In [ ]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook DB
conn = sqlite3.connect("chinook.db")

# Load required tables
invoices = pd.read_sql("SELECT InvoiceId, CustomerId FROM invoices", conn)
invoice_items = pd.read_sql("SELECT InvoiceLineId, InvoiceId, TrackId FROM invoice_items", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)

conn.close()

# Step 2: Map invoice lines to albums
invoice_tracks = invoice_items.merge(tracks, on="TrackId", how="left")

# Step 3: Total tracks per album
album_track_counts = tracks.groupby("AlbumId")["TrackId"].nunique().reset_index()
album_track_counts.rename(columns={"TrackId": "AlbumTotalTracks"}, inplace=True)

# Step 4: Tracks bought per album within each invoice
invoice_album_counts = (
    invoice_tracks.groupby(["InvoiceId", "AlbumId"])["TrackId"]
    .nunique()
    .reset_index()
    .rename(columns={"TrackId": "TracksBought"})
)

# Merge with album totals
invoice_album_counts = invoice_album_counts.merge(album_track_counts, on="AlbumId", how="left")

# Step 5: Mark full album purchases
invoice_album_counts["IsFullAlbum"] = invoice_album_counts["TracksBought"] == invoice_album_counts["AlbumTotalTracks"]

# Step 6: Identify if each customer ever bought a full album
invoice_with_customer = invoices.merge(invoice_album_counts[["InvoiceId", "IsFullAlbum"]], on="InvoiceId", how="left")

customer_pref = (
    invoice_with_customer.groupby("CustomerId")["IsFullAlbum"]
    .max()  # True if at least one full album
    .reset_index()
)

# Step 7: Classify customers
customer_pref["Preference"] = customer_pref["IsFullAlbum"].apply(lambda x: "Album" if x else "Track")

# Step 8: Calculate percentage
total_customers = customer_pref.shape[0]
track_customers = (customer_pref["Preference"] == "Track").sum()
percentage_track_pref = (track_customers / total_customers) * 100

print(f"Percentage of customers who prefer individual tracks: {percentage_track_pref:.2f}%")

# Optional: See counts
print(customer_pref["Preference"].value_counts())


In [ ]:
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).

In [ ]:
import sqlite3
import pandas as pd

# Step 1: Connect to Chinook DB
conn = sqlite3.connect("chinook.db")

# Load required tables
invoices = pd.read_sql("SELECT InvoiceId, CustomerId FROM invoices", conn)
invoice_items = pd.read_sql("SELECT InvoiceLineId, InvoiceId, TrackId FROM invoice_items", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)

conn.close()

# Step 2: Map invoice lines to albums
invoice_tracks = invoice_items.merge(tracks, on="TrackId", how="left")

# Step 3: Total tracks per album
album_track_counts = tracks.groupby("AlbumId")["TrackId"].nunique().reset_index()
album_track_counts.rename(columns={"TrackId": "AlbumTotalTracks"}, inplace=True)

# Step 4: Tracks bought per album within each invoice
invoice_album_counts = (
    invoice_tracks.groupby(["InvoiceId", "AlbumId"])["TrackId"]
    .nunique()
    .reset_index()
    .rename(columns={"TrackId": "TracksBought"})
)

# Merge with album totals
invoice_album_counts = invoice_album_counts.merge(album_track_counts, on="AlbumId", how="left")

# Step 5: Mark full album purchases
invoice_album_counts["IsFullAlbum"] = invoice_album_counts["TracksBought"] == invoice_album_counts["AlbumTotalTracks"]

# Step 6: Map to customers
invoice_with_customer = invoices.merge(
    invoice_album_counts[["InvoiceId", "IsFullAlbum"]],
    on="InvoiceId",
    how="left"
)

# Step 7: Determine each customer’s preference
customer_pref = (
    invoice_with_customer.groupby("CustomerId")["IsFullAlbum"]
    .max()  # True if at least one full album purchased
    .reset_index()
)

customer_pref["Preference"] = customer_pref["IsFullAlbum"].apply(lambda x: "Album" if x else "Track")

# Step 8: Summary percentages
summary = customer_pref["Preference"].value_counts(normalize=True) * 100
summary = summary.reset_index()
summary.columns = ["Preference", "Percentage"]

print(summary)
